In [37]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import datetime
import openpyxl

In [38]:
# 엑셀 파일 불러오기
filename = 'Wadiz_Pet0.xlsx'
data = pd.read_excel(filename)

In [39]:
# 1. 목표금액과기간에서 목표금액과 펀딩시작날짜, 펀딩마감날짜 추가
# 2. 펀딩마감날짜부터 배송까지 기간 (마감배송차이) 추가
# 3. 펀딩 시작 요일 추가


목표금액 = []
펀딩시작날짜 = []
펀딩마감날짜 = []
배송시작날짜 = []
마감배송차이 = []

for _ in range(0,len(data)):
    split_data = data.loc[_][['목표금액과기간']].str.split(' ')
    ls = split_data['목표금액과기간']
    
    # 목표금액 추출
    price = ls[2].replace(",","").replace("원","")
    목표금액.append(int(price))

    # 펀딩 시작, 마감 날짜 추출
    start_date, end_date = ls[8].split('-')
    펀딩시작날짜.append(start_date)
    펀딩마감날짜.append(end_date)
    
    # 첫 배송날짜 추출
    deliver_date = data.loc[_][['배송시작날짜']].str.split('/')
    배송시작날짜.append(deliver_date[0][0].replace(" ",""))
    
    # 첫 배송날짜와 펀딩마감 날짜의 차이 일수 계산
    oneDatetime = datetime.datetime.strptime(펀딩마감날짜[_], '%Y.%m.%d')
    twoDatetime = datetime.datetime.strptime(배송시작날짜[_], '%Y.%m.%d')
    result = twoDatetime - oneDatetime
    마감배송차이.append(result.days)
    
    
# dataframe에 추가 
data['목표금액'] = 목표금액
#ata['펀딩시작날짜'] = pd.to_datetime(펀딩시작날짜)
#ata['펀딩마감날짜'] = pd.to_datetime(펀딩마감날짜)
data['펀딩시작날짜'] = 펀딩시작날짜
data['펀딩마감날짜'] = 펀딩마감날짜
data['마감배송차이'] = 마감배송차이


# 펀딩 시작 요일 추가
펀딩시작날짜 = pd.to_datetime(펀딩시작날짜)
data['펀딩시작요일'] = 펀딩시작날짜.strftime('%a')

# 펀딩 성공 여부 ( '성공' vs '실패')
data['펀딩성공여부'] = data['달성률'].apply(lambda x: '성공' if x>99 else '실패')

# 앵콜펀딩 여부 ('앵콜' vs '최초')
data['앵콜펀딩여부'] = data['제목'].apply(lambda x: '앵콜' if  '앵콜' in x else '최초')

In [40]:
filename = filename.replace(".xlsx","_전처리.xlsx")
data.to_excel(filename)